# Display Dynamic Tab

#### Imports

In [15]:
from autotab.TabPrediction import load_model_and_weights
from autotab.TabPrediction import load_x_new
from autotab.TabPrediction import make_full_tab
import autotab.TabPrediction as tp

## Build model and load weights

In [2]:
model = load_model_and_weights()

Using Google cloud buckets
BUCKET_NAME wagon-data-737-sadriwala
DATA_BUCKET_FOLDER data/spec_repr
gs://wagon-data-737-sadriwala/data/spec_repr/
gs://wagon-data-737-sadriwala/saved/
getting ids from gs://wagon-data-737-sadriwala/data/spec_repr/id.csv


2021-11-30 14:20:35.485981: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-30 14:20:35.486042: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-30 14:20:35.486065: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-GP5BU3LN): /proc/driver/nvidia/version does not exist
2021-11-30 14:20:35.486425: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Load music file and get prediction

In [3]:
filename = '../raw_data/experimentmono.wav'
x_new = load_x_new(filename)

/mnt/d/data-science/le-wagon/autotab/autotab/TabDataReprGen.py:182: WavFileWarning: Chunk (non-data) not understood, skipping it.
  self.sr_original, data = wavfile.read(file_audio)


In [4]:
y_pred = model.predict(x_new)

## Get expanded tabs from make_full_tab

In [7]:
full_tab = make_full_tab(y_pred,len(y_pred))

/mnt/d/data-science/le-wagon/autotab/autotab/TabPrediction.py:51: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  tablature[frame_idx] = fret


In [32]:
full_tab

,0,1,2,3,4,5,6,7,8,9,...,701,702,703,704,705,706,707,708,709,710
E,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
A,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
D,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
G,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
B,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
e,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [24]:
full_tab.loc[:,16:30]

,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30
E,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
A,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
D,-1,-1,-1,-1,-1,-1,-1,-1,-1,5,5,5,5,-1,-1
G,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
B,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,8,8,8,8,8
e,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [17]:
tp.print_tabs(full_tab)

e|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|-------|
B|----------------|---------8888888|8888888----88888|----------------|---------------8|8888888888888888|----------------|----------------|----------------|----------------|----------------|----888888888888|88888888888888--|----------------|-

## Make an empty Tablature Dynamic_tab

In [54]:
dynamic_tab = tp.make_empty_tab()

## Make a counter to count columns written

In [55]:
col_write = 0

## Make a | column. Also for every 16 column, make a pipe column

## Make a counter for columns read

In [56]:
col_read=0

## Starting with 0 column, check UPTO next 4 columns, every time.increasing column read counter

In [57]:
full_tab[0]

E   -1
A   -1
D   -1
G   -1
B   -1
e   -1
Name: 0, dtype: int64

In [58]:
full_tab[9]

E   -1
A   -1
D   -1
G   -1
B   -1
e   -1
Name: 9, dtype: int64

In [59]:
full_tab[0].equals(full_tab[1])


True

In [60]:
full_tab.shape[1]

711

In [61]:
col_read=0
col_write=0
n=5
total_frames = full_tab.shape[1]
while col_read < total_frames-1: #keep doing until you reach the last frame
    changed = False # set changed as false
    max_col = col_read+n # this is the window from current col + n cols ahead 5/0
    while ((not changed) and (col_read < max_col) and (col_read < total_frames-1)):  # only if not changed and full window not checked
        if not full_tab[col_read].equals(full_tab[col_read+1]):
            # if current column doesnt equal next column, meaning change
            dynamic_tab[col_write] = full_tab[col_read] # write this col to the dynamic tab
            col_write = col_write + 1 # advance the write counter
            changed = True # set changed to true
        col_read = col_read + 1 # advance the read counter, either ways
    if not changed: # this means the entire window was checked but no change
        dynamic_tab[col_write] = full_tab[col_read-1] # so write the last entry of this window
        col_write = col_write + 1  # advance the write counter
print(col_write)
print(col_read)
#need to take care of the last column
if not dynamic_tab[col_write].equals(full_tab[col_read]):#if last written doesnt equal to last col in full_tab
    dynamic_tab[col_write + 1] = full_tab[col_read] #write this column


157
710


/tmp/ipykernel_21133/310387773.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dynamic_tab[col_write] = full_tab[col_read-1] # so write the last entry of this window
/tmp/ipykernel_21133/310387773.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dynamic_tab[col_write] = full_tab[col_read] # write this col to the dynamic tab


KeyError: 157

In [ ]:
full_tab.loc[:, 0:15]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
E,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
A,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
D,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
G,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
B,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
e,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [62]:
dynamic_tab

,0,1,2,3,4,5,6,7,8,9,...,147,148,149,150,151,152,153,154,155,156
E,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
A,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
D,-1,-1,-1,-1,-1,5,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
G,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
B,-1,-1,-1,-1,-1,8,8,8,-1,8,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
e,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [64]:
print_tabs(dynamic_tab)

e|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|-------------|
B|-----888-8------|-8888-----------|-------88888888-|-----8888-------|-----------8888-|8888-88-8888888-|----------------|----------------|----------------|-------------|
G|----------------|--------555-----|----------------|-----------7777-|----5-----------|----------------|----------------|----------------|----7777---55555|55-----------|
D|-----5----------|----------------|--------5----5--|----------------|------------5---|-5--------------|----------------|----------------|----------------|-------------|
A|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|-------------|
E|----------------|----------------|----------------|----------------|----------------|----------------|----------------|----------------|------------

## If there is a change, add current column to dynamic tab, and redo loop at current column read counter